In [ ]:
import glob
import pandas as pd

chr_hg38 = dict()
with open("hg38_chrom.txt", 'r') as inp:
    for line in inp:
        line = line.strip().split('\t')
        chr_hg38[line[1]] = int(line[0])

chr_mm10 = dict()
with open("mm10_chrom.txt", 'r') as inp:
    for line in inp:
        line = line.strip().split('\t')
        chr_mm10[line[1]] = int(line[0])


for file in glob.glob("./triadics/radiclNPM_triads.bed"):
    print(file)
    df = pd.read_table(file, names=["dna_chr", "rna_chr", "RNA", "RNA_type"])
    df["weight"] = df["dna_chr"].apply(lambda x: 10 ** 3 / chr_mm10[x])
    df["cis"] = df["dna_chr"] == df["rna_chr"]
    fname = input()
    df.groupby(["RNA", "RNA_type", "cis"])["weight"].sum().reset_index().to_csv(f"{fname}.tab", sep='\t', index=False, header=True)


In [ ]:
import os

#os.mkdir("normalized")

for file in glob.glob("./*ratios*tab*"):
    df = pd.read_table(file)
    df["cis"] = df["cis"].apply(lambda x: "cis" if x else "trans")
    df = df.pivot(index = ["RNA", "RNA_type"], columns="cis", values="weight").reset_index()
    df["ratio"] = (df["cis"] + 0) / (df["trans"])
    df.iloc[:, [0,1,4]].to_csv(f"normalized/{file}", sep='\t', index=False, header=True)

In [ ]:
from scipy.stats import wilcoxon
dfl = dict()
pvals = []
for file in glob.glob("normalized/[rg]*tab"):
    triads = file.split('/')
    triads[1] = "triads_" + triads[1]
    triads = '/'.join(triads)
    
    rd = pd.read_table(file)
    triad = pd.read_table(triads)
    df = rd.merge(triad, on=["RNA", "RNA_type"], suffixes=["_RD", "_TRIAD"]).dropna()
    dfl[file.split('/')[-1].split('.')[0].split('_')[0]] = df
    res = wilcoxon(df["ratio_RD"], df["ratio_TRIAD"])
    print(f"{file.split('/')[-1].split('.')[0].split('_')[0]}\t{res[0]}\t{res[1]}")
    pvals.append(res[1])

In [ ]:
from statsmodels.stats.multitest import fdrcorrection as FDR

FDR(pvals)

In [ ]:
l = []
with open("top100_rnas/redc.txt", 'r') as fin:
    for line in fin:
        l.append(line.strip().split(' ')[-1])

In [ ]:
tmp = dfl["redc"]
tmp

In [83]:
tmp[tmp.RNA.isin(l)]

,RNA,RNA_type,ratio_RD,ratio_TRIAD
10,1840_HUVEC,vlinc,0.193256,0.130892
14,2125_K562,vlinc,0.177960,0.120279
35,2286_K562,vlinc,0.213551,0.170541
61,AC005208.1,lncRNA,0.168325,0.136384
78,AC012494.1,lncRNA,0.182849,0.064323
86,AC020687.1,lncRNA,0.290336,0.163065
101,AC097528.1,lncRNA,0.194500,0.031450
156,AL390334.1,transcribed_processed_pseudogene,0.252462,0.090108
158,AL445255.1,lncRNA,0.271939,0.073506
163,AL713998.1,lncRNA,0.074473,0.078996
